In [1]:
import os

os.environ['CUDA_VISIBLE_DEVICES'] = '2'

In [2]:
from whisper_conv import WhisperForConditionalGeneration
from transformers import AutoFeatureExtractor, AutoTokenizer, AutoProcessor
import librosa
import torch

torch.set_grad_enabled(False)

In [3]:
WhisperForConditionalGeneration.register_for_auto_class("AutoModel")

In [4]:
!ls whisperconv

checkpoint-41000  checkpoint-41500  checkpoint-42000  runs


In [5]:
model = WhisperForConditionalGeneration.from_pretrained(
    'whisperconv/checkpoint-42000', torch_dtype = 'auto')

In [6]:
_ = model.cuda()

In [7]:
feature_extractor = AutoFeatureExtractor.from_pretrained('openai/whisper-large-v3-turbo')
tokenizer = AutoTokenizer.from_pretrained('openai/whisper-large-v3-turbo')
processor = AutoProcessor.from_pretrained('openai/whisper-large-v3-turbo')

In [8]:
from streaming import MDSWriter, LocalDataset

dataset = LocalDataset('mosaic-stt-include-malaysian')

In [9]:
import IPython.display as ipd
ipd.Audio(dataset[20000]['audio_filename'])

In [10]:
dataset[-100003]['audio_filename'], dataset[-100002]['audio_filename']

('audio/fr/dev/audio/common_voice_fr_19722301.mp3',
 'audio/fr/dev/audio/common_voice_fr_19722302.mp3')

In [11]:
i = 20000
t = dataset[i]['text']
y, sr = librosa.load(dataset[i]['audio_filename'], sr = feature_extractor.sampling_rate)
tokenized = tokenizer(dataset[i]['text'], add_special_tokens = False)['input_ids']
input_ids = []
for k in tokenized:
    input_ids.append(k)
    if k == 50364:
        break
features = feature_extractor([y], return_tensors = 'pt', return_attention_mask = True)
features['decoder_input_ids'] = torch.tensor(input_ids)[None]
for k in features.keys():
    features[k] = features[k].cuda()

generate_kwargs = dict(
    **features,
    max_new_tokens=1024,
    temperature=0.0,
    do_sample=False
)
generation_output = model.generate(**generate_kwargs)
tokenizer.decode(generation_output[0]), t

It is strongly recommended to pass the `sampling_rate` argument to `WhisperFeatureExtractor()`. Failing to do so can result in silent errors that might be hard to debug.
The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


("<|startoftranscript|><|en|><|transcribe|><|notimestamps|> But wait lah, I see my area's results online. But official portal still blank. Which one true one?<|endoftext|>",
 "<|startoftranscript|><|en|><|transcribe|><|notimestamps|> But wait la, I see my area's results online. But official portal still blank. Which one true one?<|endoftext|>")

In [12]:
y, sr = librosa.load('218757.mp3', sr = feature_extractor.sampling_rate)
input_ids = tokenizer(
    '<|startoftranscript|><|ms|><|transcribe|><|notimestamps|>', 
    add_special_tokens = False, return_tensors = 'pt')['input_ids']
features = feature_extractor([y], return_tensors = 'pt', return_attention_mask = True)
features['decoder_input_ids'] = input_ids
for k in features.keys():
    features[k] = features[k].cuda()

generate_kwargs = dict(
    **features,
    max_new_tokens=1024,
    temperature=0.1,
    do_sample=True
)
generation_output = model.generate(**generate_kwargs)
tokenizer.decode(generation_output[0])

It is strongly recommended to pass the `sampling_rate` argument to `WhisperFeatureExtractor()`. Failing to do so can result in silent errors that might be hard to debug.


'<|startoftranscript|><|ms|><|transcribe|><|notimestamps|> Siewkiong sebelum ini memenangi kerusi itu dengan hanya dua-ratus tiga puluh lapan undi ketika beliau menewaskan calon DAP, Diana Sophia Muhammad Daud, pada pilihan raya kecil kurungan terbuka PRK kurungan tertutup dua pul pul pul pul pul pul pul pulap-p.<|endoftext|>'

In [13]:
model.push_to_hub('mesolitica/whisper-conv-large-v3-turbo')

[2025-08-01 07:28:26,500] [INFO] [real_accelerator.py:239:get_accelerator] Setting ds_accelerator to cuda (auto detect)


/usr/bin/ld: cannot find -laio
collect2: error: ld returned 1 exit status
/usr/bin/ld: /usr/local/cuda/lib64/libcufile.so: undefined reference to `dlopen'
/usr/bin/ld: /usr/local/cuda/lib64/libcufile.so: undefined reference to `dlclose'
/usr/bin/ld: /usr/local/cuda/lib64/libcufile.so: undefined reference to `dlerror'
/usr/bin/ld: /usr/local/cuda/lib64/libcufile.so: undefined reference to `dlsym'
collect2: error: ld returned 1 exit status


Uploading...:   0%|          | 0.00/3.24G [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/mesolitica/whisper-conv-large-v3-turbo/commit/afb533f3bf80aa130006d7f8322c52e22f2496c2', commit_message='Upload WhisperForConditionalGeneration', commit_description='', oid='afb533f3bf80aa130006d7f8322c52e22f2496c2', pr_url=None, repo_url=RepoUrl('https://huggingface.co/mesolitica/whisper-conv-large-v3-turbo', endpoint='https://huggingface.co', repo_type='model', repo_id='mesolitica/whisper-conv-large-v3-turbo'), pr_revision=None, pr_num=None)

In [16]:
tokenizer.push_to_hub('mesolitica/whisper-conv-large-v3-turbo')

No files have been modified since last commit. Skipping to prevent empty commit.


CommitInfo(commit_url='https://huggingface.co/mesolitica/whisper-conv-large-v3-turbo/commit/afb533f3bf80aa130006d7f8322c52e22f2496c2', commit_message='Upload tokenizer', commit_description='', oid='afb533f3bf80aa130006d7f8322c52e22f2496c2', pr_url=None, repo_url=RepoUrl('https://huggingface.co/mesolitica/whisper-conv-large-v3-turbo', endpoint='https://huggingface.co', repo_type='model', repo_id='mesolitica/whisper-conv-large-v3-turbo'), pr_revision=None, pr_num=None)

In [17]:
feature_extractor.push_to_hub('mesolitica/whisper-conv-large-v3-turbo')

No files have been modified since last commit. Skipping to prevent empty commit.


CommitInfo(commit_url='https://huggingface.co/mesolitica/whisper-conv-large-v3-turbo/commit/afb533f3bf80aa130006d7f8322c52e22f2496c2', commit_message='Upload feature extractor', commit_description='', oid='afb533f3bf80aa130006d7f8322c52e22f2496c2', pr_url=None, repo_url=RepoUrl('https://huggingface.co/mesolitica/whisper-conv-large-v3-turbo', endpoint='https://huggingface.co', repo_type='model', repo_id='mesolitica/whisper-conv-large-v3-turbo'), pr_revision=None, pr_num=None)

In [18]:
processor.push_to_hub('mesolitica/whisper-conv-large-v3-turbo')

CommitInfo(commit_url='https://huggingface.co/mesolitica/whisper-conv-large-v3-turbo/commit/53e4ad3204e72788aab2700afa4e7bc73c14c256', commit_message='Upload processor', commit_description='', oid='53e4ad3204e72788aab2700afa4e7bc73c14c256', pr_url=None, repo_url=RepoUrl('https://huggingface.co/mesolitica/whisper-conv-large-v3-turbo', endpoint='https://huggingface.co', repo_type='model', repo_id='mesolitica/whisper-conv-large-v3-turbo'), pr_revision=None, pr_num=None)